<a href="https://colab.research.google.com/github/elizabethzhu1/critic-user-llm-model/blob/main/critic_%2B_user_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00


In [3]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import random
import numpy as np
import openai

openai.api_key = input("Enter your OpenAI API key: ")

USER_MODEL = "gpt-3.5-turbo"
CRITIC_MODEL = "gpt-3.5-turbo"

In [5]:
class Critic:

  def __init__(self, constraints):
    self.constraints = constraints

  def giveFeedback(self, response):

    feedback = openai.ChatCompletion.create(
        model = CRITIC_MODEL,
        messages = [
            {"role": "system", "content": "You are to evaluate and give feedback on a response that a 'User' has generated, specifically evaluating whether it fits each of the following set of criteria: " + str(self.constraints) + ". Please only give feedback on whether the response follows the criteria in this array."},
            {"role": "system", "content": """In your evaluation, you are to include two things.
            First, state a numeric rating for the response, where the numerator is the number of criteria
            that the response successfully fulfills and the denominator is the number of total criteria.
            For example, a possible rating could take the form: 3/5, to indicate only 3 of the total
            criteria being met. Please carefully check and specifically state whether each constraint is being met.
            Second, give criticism on how the response could improve to meet one of the criteria.
            For example, a possible criticism could take the form: "The response is not written
            in a depressed tone. Please adjust the tone accordingly, to meet this criteria.
            Do not give an improved response, simply provide feedback for the user.
            """},
            # {"role": "user", "content": "Evaluate the following response based on whether it fulfills the following constraints: ['written in a pirate accent', 'under 50 words', 'logical']. The response you are to evaluate is: The book is better than the movie for a number of reasons. The book is more detailed and creates a richer picture in the reader's mind. The book also allows the reader to get to know the characters better and understand their motivations. The movie leaves out many important details from the book and simplifies the characters' motivations."},
            # {"role": "assistant", "content": "This response gets a rating of 1/3. This is because it meets the third constraint of being logical, but is not written in a pirate accent and is not under 50 words. To improve this response to better fit the given constraints, we should adjust the wording to be in a pirate accent and cut it down by 8 words."},
            {"role": "user", "content": str(response)}
        ]
    )

    feedback = feedback.choices[0].message['content'].strip()

    return feedback


In [6]:
sample_prompt = " Write a paragraph arguing why the book is better than the movie. "

sample_response = openai.ChatCompletion.create(
    model = USER_MODEL,
    messages = [
        {"role": "user", "content": sample_prompt}
    ]
  )

sample_response = sample_response.choices[0].message['content'].strip()

sample_constraints = ["Constraint 1: The response should be written in a pirate accent", "Constraint 2: The response should be less than 50 words",
                      "Constraint 3: The response should be logical",  "Constraint 4: The response should use a specific example to illustrate the argument",
                      "Constraint 5: The response should use the word great 5 times"]

critic = Critic(sample_constraints)
feedback = critic.giveFeedback(sample_response)

print("Sample Response: ", sample_response, "\n")
print("Critic Feedback: ", feedback)

Sample Response:  Books are often better than their movie adaptations for several reasons. Firstly, books have the advantage of being able to provide more detailed and complex storytelling. They offer the luxury of exploring the characters' thoughts, emotions, and motivations in a way that movies simply cannot. Additionally, books allow readers to engage their imagination and create their own mental pictures of the story, making for a more personal and immersive experience. Furthermore, books are not bound by time constraints, allowing for more subplots and additional character development that often get sacrificed in movie adaptations. Lastly, books are the original source, often written by talented and skilled authors who spend years crafting their stories. The movie adaptation, on the other hand, is just someone else's interpretation, and may not fully capture the essence and depth of the original material. In the end, the book offers a richer and more fulfilling experience than the

In [ ]:
critic = Critic(sample_constraints)
feedback = critic.giveFeedback(sample_response)

print("Sample Response: ", sample_response, "\n")
print("Critic Feedback: ", feedback)

Sample Response:  When it comes to the debate of whether a book is better than its movie adaptation, there are several compelling reasons to opt for the printed word. Firstly, the book allows the reader to delve deeper into the characters’ minds and emotions, providing a more comprehensive understanding of their motives and actions than a visual medium can achieve. Additionally, books elicit the personal experience of imagery, which encourages the development of imagination; each reader envisions scenarios and character appearances in a unique manner, adding a layer of richness absent from a universally presented visual portrayal. Movies, by virtue of their format, are also confined by time constraints that often necessitate the omission or truncation of critical plot points and character arcs, whereas books provide a more leisurely pace to fully explore and savor every aspect of the story. Finally, books foster a more intimate connection with the author, who can masterfully manipulate

In [ ]:
from typing import Iterable, Union
# defines a User class, which takes in the Critic's feedback and the original prompt, and generates a fitting response.
class User:

  def __init__(self, starting_prompt: str, history=None):
    # pass in an array of all previous chat responses
    if history is None:
      self.history = []
    else:
      self.history = history
    self.system_message = {
        "role": "system",
        "content": "Your task is to answer the given prompt as well as possible: " + starting_prompt
    }

  def generateResponse(self):

    message_history = [self.system_message] + self.history

    response = openai.ChatCompletion.create(
        model = USER_MODEL,
        messages = message_history
      )

    response = response.choices[0].message['content'].strip()

    return response

  def update_history(self, new_items: Union[dict, Iterable]):
    if isinstance(new_items, dict):
      self.history.append(new_items)
    else:
      self.history.extend(new_items)

In [ ]:
user = User(
    starting_prompt="Write an argument for why the book is better than the movie.",
    history=[
         {"role": "assistant", "content": "The book is better than the user because it is more detailed."},
         {"role": "user", "content": "Please improve your next response to include a specific example."}
         ]
    )
response = user.generateResponse()
print(response)

The book is often better than the movie for several reasons, one of which is the level of detail and depth that a book can offer. Books allow authors to delve into the intricate details of the characters, their thoughts, and emotions, as well as the setting and plot. This results in a richer and more immersive experience for readers as they can empathize with the characters and truly understand their motives and feelings.

Take, for example, the Harry Potter series by J.K. Rowling. In the books, Rowling intricately weaves together a magical world filled with complex characters, rich history, and intricate subplots. The level of detail and nuance that goes into developing the Wizarding World is something that simply cannot be captured in its entirety on the big screen. Readers are given insights into the backgrounds, fears, and hopes of not just the main characters but also many side characters, making it easy to get lost in the world and develop a strong emotional attachment to the sto

In [ ]:
def user_and_critic():
  history = []

  original_prompt = "Generate a list of numbered reasons for why pineapple should be on pizza."

  constraints = ["The response answers the original prompt",
                 "The response should contain exactly 3 reasons",
                 "The response should be under 75 words",
                 "The response should be written in a pirate accent",
                 "The response should be written in numbered list format"]

  user = User(starting_prompt=original_prompt)
  critic = Critic(constraints)

  for i in range(5):

    # user gives response based on previous chat history
    user_response = user.generateResponse()
    # NOTE: the word "user" is overloaded. The "user" in our metaprompt setup plays the role of
    # the "assistant" in the ChatCompletion API.
    user.update_history({
        "role": "assistant",
        "content": user_response
    })
    print("User: " + user_response + "\n")

    # based on previous user response, critic gives feedback
    critic_feedback = critic.giveFeedback(user_response)
    print("Critic: " + critic_feedback + "\n")
    user.update_history({
        "role": "user",  # the critic is the user in the ChatCompletion API,
        "content": critic_feedback
    })

user_and_critic()


User: 1. Unique flavor combination: Pineapple brings a sweet and tangy taste that perfectly complements the savory flavors of cheese, tomato sauce, and other toppings on pizza.

2. Balanced taste: The sweetness of pineapple balances out the saltiness from the cheese and other meat toppings like ham or bacon, resulting in a more well-rounded flavor profile.

3. Textural contrast: Pineapple adds a juicy, tender texture that contrasts well with the crispy crust and chewy cheese, enhancing the overall eating experience.

4. Nutritional value: Pineapple is rich in Vitamin C, manganese, and antioxidants, providing additional health benefits when included as a pizza topping.

5. Culinary creativity: The use of pineapple as a pizza topping demonstrates innovative thinking and a willingness to experiment with new flavor combinations, pushing the boundaries of traditional pizza recipes.

6. Customization: Including pineapple as a topping option allows consumers to personalize their pizzas accord